In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-dataset/heart.csv


In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/kaggle/input/heart-disease-dataset/heart.csv")
df.shape

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
sns.countplot(data=df,x='cp')
plt.show()

0 – **Typical angina** (chest pain related to decreased blood supply to the heart)

1 – **Atypical angina** (chest pain not directly related to heart)

2 – **Non-anginal pain** (not heart-related)

3 – **Asymptomatic** (no chest pain but disease may exist)

From the countplot,we can clearly see dataset covers majority of typical angina cases

In [ ]:
sns.countplot(x='cp',data=df,hue='target')
plt.show()

Patients with type 2 chest pain show higher proportion of heart diseases cases.As a result, we can assume, chest pain typically not related to heart may still indicates underlying heart problems.In contrast, doctors should not be soley rely on directly associated chest pain for diagonosing heart disease.

In [ ]:
sns.countplot(x='fbs',hue='target',data=df)


If FBS is high (e.g., >120 mg/dL in your dataset), it means blood sugar is elevated even without eating, which can indicate insulin problems or diabetes.

If FBS is normal or low, it means body is regulating blood sugar properly during fasting.

Patients with normal fasting blood sugar shows higher proportion of heart problems,whereas proportion of patient with higher fasting blood sugar indicates less possibility of heart issue.

The dataset is imbalanced for fbs. There are more cases of patient with fbs-0 than fbs-1.However, it is rare to find patients with fbs-1, since fewer people have higher fasting blood sugar compared to normal.

In [ ]:
sns.countplot(x='restecg',hue='target',data=df)
plt.show()


0 – Normal → Heart looks healthy.

1 – ST-T wave abnormality → Heart is under stress or not getting enough oxygen.

2 – Left ventricular hypertrophy → Heart muscle is thicker than normal, meaning the heart is working harder than it should.

The restecg feature is a strong predictor of heart disease.

An restecg value of 1 (ST-T wave abnormality) is strongly associated with the presence of heart disease (target=1).

An restecg value of 0 (Normal/Mild Abnormality) is more associated with the absence of heart disease (target=0).

In [ ]:
sns.countplot(x='exang', hue='target', data=df)


**Exercise Induced Angina**

1->	The patient experiences chest pain (angina) during exercise or physical exertion.

0->	The patient does not experience exercise-induced chest pain.

The figure strongly indicates that patients who do not experience chest pain during exercise have higher rate of developing heart disease.This conclude that absence of chest pain during physical exertion doesnt necessary imply patient is free from heart disease. 



In [ ]:
sns.countplot(x='ca',hue='target',data=df)


ca - number of major vessels (0–3) colored by fluoroscopy

ca counts how many of the major arteries (left anterior descending, circumflex, right coronary artery, etc.) have significant narrowing or blockage.

The higher the ca value, the more severe the coronary artery disease, because multiple vessels are affected.

#### The figure shows even with zero ca, patients have higher ratio of developing heart diseases. ####

In [ ]:
sns.countplot(x='thal',hue='target',data=df)

**1 → healthy blood flow**

**2 → part of heart has permanent damage**

**3 → part of heart is temporarily affected under stress**


**0 → missing or unknown information**

thal-2 is a strong predictor for heart disease as it contain more cases compared to other types

In constrast,thal-3 is strong indicate normal heart condition, with fewer cases of heart issues.

In [ ]:
cat_data=['cp','fbs','restecg','exang','ca','thal']
plt.figure(figsize=(10,5))
for i,feature in enumerate(cat_data):
    plt.subplot(2,3,i+1)
    sns.countplot(x=df[feature],hue=df['target'])
    plt.title(feature)
plt.tight_layout()
plt.show()

In [ ]:
sns.histplot(df['trestbps'], kde=True)
plt.title('Distribution of Resting BP')
plt.show()

In [ ]:
sns.histplot(df['chol'],kde=True)

This histogram illustrates the distribution of **serum cholesterol (chol)** levels in the dataset. The data is heavily concentrated between **200 and 300 mg/dL**, peaking around **250 mg/dL**. This indicates that a large portion of the individuals in the sample already have cholesterol levels considered high or borderline high, which is typical for a dataset used to predict heart disease. While the core of the distribution is bell-shaped, the presence of distinct, small peaks far to the right (around 400 and 550 mg/dL) shows that the dataset also includes a few patients with **extremely high and medically concerning cholesterol levels**. Overall, the high average and wide range of cholesterol values confirm this feature's relevance as a critical **risk factor** for heart disease prediction.

In [ ]:
sns.histplot(df['thalach'],kde=True)

**Most people have normal/high max heart rates, but a small number have low rates, which signals higher risk for heart disease.**

In [ ]:
sns.histplot(df['age'],kde=True)


This histogram displays the distribution of Age in the heart disease prediction dataset.

Observations
Age Range: The ages in the dataset span from around 29 up to 77, though the vast majority of individuals are between 40 and 70.

Most Common Ages (Mode): The distribution is clearly bimodal (two main peaks). The most prominent peak is around 58 to 60 years old, and a secondary, smaller peak occurs around 42 to 45 years old.

Central Tendency: The majority of the data is concentrated in the middle-aged to older adult groups, primarily between 50 and 65.

Shape: The overall shape is skewed, with the high counts concentrated towards the center-right of the graph.

In [ ]:
sns.scatterplot(x='age',y='thalach',data=df)

In [ ]:
sns.scatterplot(x='chol',y='trestbps',data=df)

In [ ]:
num_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Pairplot with target as hue
sns.pairplot(df[num_features + ['target']], hue='target', diag_kind='kde')
plt.show()

In [ ]:
sns.scatterplot(x='thalach',y='oldpeak',hue='target',data=df)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
numeric_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

X = df[numeric_features + categorical_features]
y = df['target']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

preprocessor=ColumnTransformer([
    ('num',StandardScaler(),numeric_features),
    ('cat',OneHotEncoder(drop='first'),categorical_features)
])

pipeling=Pipeline([
    ('preprocessor',preprocessor),
    ('svm',SVC(random_state=42))
])
param_grid={
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 0.01, 0.1, 1],
    'svm__kernel': ['rbf', 'poly']
}

grid_search=GridSearchCV(  pipeling, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

grid_search.fit(X_train,y_train)
best_model = grid_search.best_estimator_
best_model

In [ ]:
y_pred=best_model.predict(X_test)
print(grid_search.best_params_)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
print("Classification report ",classification_report(y_test,y_pred))

In [ ]:
train_acc = best_model.score(X_train, y_train)
test_acc = best_model.score(X_test, y_test)
print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(best_model, X, y, cv=10)
print("CV Accuracy:", scores.mean())


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


# Pipeline with Random Forest
pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(random_state=42))
])

# Hyperparameter grid for Random Forest
param_grid_rf = {
    'rf__n_estimators': [100, 200, 500],
    'rf__max_depth': [None, 5, 10],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2]
}

# Grid Search
grid_search_rf = GridSearchCV(
    pipeline_rf, 
    param_grid_rf, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=1
)

# Fit
grid_search_rf.fit(X_train, y_train)

# Best model
best_rf = grid_search_rf.best_estimator_
print("Best Random Forest parameters:", grid_search_rf.best_params_)
print("Test Accuracy:", best_rf.score(X_test, y_test))
print("Classification report ",classification_report(y_test,y_pred))
scores = cross_val_score(best_rf, X, y, cv=10)

print("Cross validation score",scores.mean())
